In [5]:
#pip install yfinance pandas numpy requests

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta

def get_risk_free_rate():
    """Fetches the 10-year Treasury yield as a proxy for the risk-free rate."""
    try:
        url = "https://api.stlouisfed.org/fred/series/observations?series_id=DGS10&api_key=YOUR_API_KEY&file_type=json"
        response = requests.get(url).json()
        rate = float(response["observations"][-1]["value"]) / 100  # Convert from % to decimal
        return rate
    except:
        return 0.02  # Default to 2% if API fails

def fetch_options_data(ticker):
    """Fetches real-time options data for the given stock ticker from Yahoo Finance."""
    stock = yf.Ticker(ticker)
    current_price = stock.history(period="1d")["Close"].iloc[-1]
    
    # Get first available expiration date
    expiry_dates = stock.options
    if not expiry_dates:
        print(f"No options available for {ticker}")
        return None
    
    expiry = expiry_dates[0]  # Earliest expiration date
    options_chain = stock.option_chain(expiry)
    
    # Select a near-the-money call option
    calls = options_chain.calls
    calls["moneyness"] = abs(calls["strike"] - current_price)
    selected_call = calls.nsmallest(1, "moneyness").iloc[0]
    
    # Compute time to maturity (T) in years
    expiry_date = datetime.strptime(expiry, "%Y-%m-%d")
    today = datetime.today()
    T = (expiry_date - today).days / 365.0  # Convert days to years
    
    # Get risk-free rate
    r = get_risk_free_rate()
    
    # Extract required parameters
    option_data = {
        "Stock Price (S)": current_price,
        "Strike Price (K)": selected_call["strike"],
        "Time to Maturity (T) (years)": T,
        "Risk-Free Rate (r)": r,
        "Implied Volatility (σ)": selected_call["impliedVolatility"],
        "Option Price": selected_call["lastPrice"],
        "Expiration Date": expiry
    }
    
    return option_data

# Example Usage
ticker = "AAPL"  # Change to any stock symbol
option_data = fetch_options_data(ticker)

if option_data:
    print("\n🔹 Real-World Option Data for Black–Scholes Model:")
    for key, value in option_data.items():
        print(f"{key}: {value}")
        
## If T = 0 you need to pick a value for it


🔹 Real-World Option Data for Black–Scholes Model:
Stock Price (S): 237.3000030517578
Strike Price (K): 237.5
Time to Maturity (T) (years): 0.0
Risk-Free Rate (r): 0.02
Implied Volatility (σ): 0.3789124609375
Option Price: 1.68
Expiration Date: 2025-02-28


In [2]:
import tensorflow as tf
import tensorflow_probability as tfp

# Standard Normal CDF
normal = tfp.distributions.Normal(loc=0.0, scale=1.0)

def compute_greeks(S, K, T, r, sigma, option_type="call"):
    """Compute Black-Scholes price and Greeks (Delta, Gamma, Vega, Theta, Rho)."""

    T = tf.maximum(T, 1e-6)  # Avoid division by zero
    d1 = (tf.math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * tf.sqrt(T))
    d2 = d1 - sigma * tf.sqrt(T)

    # Standard normal probabilities
    N_d1 = normal.cdf(d1)
    N_d2 = normal.cdf(d2)

    # Greeks calculations
    price = None
    delta = None
    gamma = normal.prob(d1) / (S * sigma * tf.sqrt(T))
    vega = S * normal.prob(d1) * tf.sqrt(T)
    theta = None
    rho = None

    if option_type == "call":
        price = S * N_d1 - K * tf.exp(-r * T) * N_d2
        delta = N_d1
        theta = (-S * normal.prob(d1) * sigma / (2 * tf.sqrt(T))) - r * K * tf.exp(-r * T) * N_d2
        rho = K * T * tf.exp(-r * T) * N_d2
    elif option_type == "put":
        price = K * tf.exp(-r * T) * (1 - N_d2) - S * (1 - N_d1)
        delta = N_d1 - 1
        theta = (-S * normal.prob(d1) * sigma / (2 * tf.sqrt(T))) + r * K * tf.exp(-r * T) * (1 - N_d2)
        rho = -K * T * tf.exp(-r * T) * (1 - N_d2)

    return {
        "Price": price.numpy(),
        "Delta": delta.numpy(),
        "Gamma": gamma.numpy(),
        "Vega": vega.numpy(),
        "Theta": theta.numpy(),
        "Rho": rho.numpy()
    }

# Example values (Replace these with real-world data)
S = 237.45  # Stock price
K = 237.5   # Strike price
T = 0.119   # Time to expiration in years
r = 0.02    # Risk-free interest rate
sigma = 0.419  # Implied volatility

# Compute Greeks
greeks = compute_greeks(S, K, T, r, sigma, option_type="call")

print("\n🔹 Black–Scholes Model Results Using Real-World Data:")
for key, value in greeks.items():
    print(f"{key}: {value}")


🔹 Black–Scholes Model Results Using Real-World Data:
Price: 13.924186706542969
Delta: 0.5347752571105957
Gamma: 0.011579671874642372
Vega: 32.55378723144531
Theta: -59.57224655151367
Rho: 13.453926086425781


In [3]:
print(f"S: {S}, K: {K}, T: {T}, r: {r}, sigma: {sigma}")

S: 237.45, K: 237.5, T: 0.119, r: 0.02, sigma: 0.419


In [4]:
import numpy as np
from scipy.stats import norm

def black_scholes_numpy(S, K, T, r, sigma, option_type="call"):
    """ Computes Black-Scholes price and Greeks using NumPy/SciPy. """

    # Avoid division by zero
    T = max(T, 1e-6)
    sigma = max(sigma, 1e-6)

    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    # Greeks
    delta = norm.cdf(d1) if option_type == "call" else norm.cdf(d1) - 1
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * np.sqrt(T) * norm.pdf(d1)

    # Option Price
    if option_type == "call":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * (1 - norm.cdf(d2)) - S * (1 - norm.cdf(d1))

    return {
        "Price": price,
        "Delta": delta,
        "Gamma": gamma,
        "Vega": vega
    }

In [5]:
# Test parameters (from your previous result)
S = 237.4499969482422
K = 237.5
T = 0.119
r = 0.02
sigma = 0.41919526123046874
option_type = "call"

# Compute Greeks using TensorFlow
tf_greeks = compute_greeks(S, K, T, r, sigma, option_type)

# Compute Greeks using NumPy/SciPy
np_greeks = black_scholes_numpy(S, K, T, r, sigma, option_type)

# Print comparison
print("\n🔹 Black–Scholes Model Benchmark Comparison")
for key in tf_greeks.keys():
    print(f"{key}: TensorFlow = {tf_greeks[key]:.6f}, SciPy = {np_greeks[key]:.6f}")


🔹 Black–Scholes Model Benchmark Comparison
Price: TensorFlow = 13.930534, SciPy = 13.930538
Delta: TensorFlow = 0.534786, SciPy = 0.534786
Gamma: TensorFlow = 0.011574, SciPy = 0.011574
Vega: TensorFlow = 32.553711, SciPy = 32.553714


KeyError: 'Theta'

🔹 Interpretation of the Greeks
Price: The Black-Scholes estimated option price.
Delta (0.53): The option gains ~$0.53 for every $1 increase in stock price.
Gamma (0.0116): Delta’s rate of change is small, which is expected for options close to at-the-money.
Vega (32.55): The option’s sensitivity to volatility changes is high—a 1% increase in volatility changes the price by ~$32.55.
Theta (-59.57): The option is losing ~$59.57 per day due to time decay (negative since it's an option).
Rho (13.45): A 1% increase in interest rates raises the price by ~$13.45.